# Basic Examples


## Dask Arrays

In [11]:
import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [1]:
import dask.dataframe as dd
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 10,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55273,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:55297,Total threads: 2
Dashboard: http://127.0.0.1:55298/status,Memory: 3.20 GiB
Nanny: tcp://127.0.0.1:55278,


### Create Random array
This creates a 10000x10000 array of random numbers, represented as many numpy arrays of size 1000x1000 (or smaller if the array cannot be divided evenly). In this case there are 100 (10x10) numpy arrays of size 1000x1000.

In [4]:
import dask.array as da
x = da.random.random((10000, 10000), chunks=(1000, 1000))
x

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

In [7]:
x.compute()

array([[0.01634624, 0.32558483, 0.64571269, ..., 0.52923954, 0.94409129,
        0.29575452],
       [0.80795311, 0.53850723, 0.2213644 , ..., 0.01177475, 0.47993087,
        0.26007514],
       [0.60914018, 0.74377922, 0.37742864, ..., 0.04584268, 0.71122608,
        0.09491565],
       ...,
       [0.47240803, 0.07804243, 0.05392034, ..., 0.03695387, 0.45432321,
        0.30066794],
       [0.34659286, 0.86176731, 0.91765746, ..., 0.81095883, 0.45018425,
        0.18814358],
       [0.81495558, 0.68104896, 0.70027754, ..., 0.42491238, 0.79083665,
        0.1861335 ]])

In [12]:
# Use NumPy syntax as usual
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z

dask.array<mean_agg-aggregate, shape=(5000,), dtype=float64, chunksize=(500,), chunktype=numpy.ndarray>

In [13]:
z.compute()
z.shape

array([1.0036755 , 0.99734762, 0.9960023 , ..., 0.99556423, 1.00513176,
       0.99794343])

(5000,)

### Persist data in memory
If you have the available RAM for your dataset then you can persist data in memory.

In [14]:
y = y.persist()
%time y[0, 0].compute()

CPU times: user 101 ms, sys: 8.93 ms, total: 110 ms
Wall time: 151 ms


0.03269248026511029

In [15]:
%time y.sum().compute()


CPU times: user 75.6 ms, sys: 6.08 ms, total: 81.7 ms
Wall time: 96.3 ms


99986490.73732442

# Dask Bags
Dask Bag implements operations like map, filter, groupby and aggregations on collections of Python objects. It does this in parallel and in small memory using Python iterators. It is similar to a parallel version of itertools or a Pythonic version of the PySpark RDD.

Dask Bags are often used to do simple preprocessing on log files, JSON records, or other user defined Python objects.

## Create Random Data
We create a random set of record data and store it to disk as many JSON files. This will serve as our data for this notebook.



In [19]:
import dask
import json
import os

os.makedirs('data', exist_ok=True)              # Create data/ directory

b = dask.datasets.make_people()                 # Make records of people
b.map(json.dumps).to_textfiles('data/*.json')   # Encode as JSON, write to disk

['/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/0.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/1.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/2.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/3.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/4.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/5.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/6.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/7.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/8.json',
 '/Users/parsanemati/Yandex.Disk.localized/git

In [20]:
# pip install mimesis

In [21]:
!head -n 2 data/0.json

{"age": 22, "name": ["Roger", "Whitaker"], "occupation": "Bill Poster", "telephone": "309.151.9537", "address": {"address": "195 Bergen Stravenue", "city": "Batavia"}, "credit-card": {"number": "4050 7093 7680 9254", "expiration-date": "10/22"}}
{"age": 19, "name": ["Olen", "Shepard"], "occupation": "Show Jumper", "telephone": "+1-(701)-259-4365", "address": {"address": "1229 Finley Stravenue", "city": "Laconia"}, "credit-card": {"number": "4187 5843 8083 9670", "expiration-date": "04/24"}}


In [27]:
import dask.bag as db
import json

b = db.read_text('data/*.json').map(json.loads)
b.take(3)

({'age': 22,
  'name': ['Roger', 'Whitaker'],
  'occupation': 'Bill Poster',
  'telephone': '309.151.9537',
  'address': {'address': '195 Bergen Stravenue', 'city': 'Batavia'},
  'credit-card': {'number': '4050 7093 7680 9254',
   'expiration-date': '10/22'}},
 {'age': 19,
  'name': ['Olen', 'Shepard'],
  'occupation': 'Show Jumper',
  'telephone': '+1-(701)-259-4365',
  'address': {'address': '1229 Finley Stravenue', 'city': 'Laconia'},
  'credit-card': {'number': '4187 5843 8083 9670',
   'expiration-date': '04/24'}},
 {'age': 29,
  'name': ['Art', 'Weeks'],
  'occupation': 'Publishing Manager',
  'telephone': '1-793-335-3530',
  'address': {'address': '1058 Ashburton Freeway', 'city': 'Hialeah'},
  'credit-card': {'number': '3462 831917 58783', 'expiration-date': '05/19'}})

### Map, Filter, Aggregate
We can process this data by filtering out only certain records of interest, mapping functions over it to process our data, and aggregating those results to a total value.

In [28]:
b.filter(lambda record: record['age'] > 30).take(2)  # Select only people over 30


({'age': 45,
  'name': ['Moshe', 'Pope'],
  'occupation': 'Artexer',
  'telephone': '148-805-6650',
  'address': {'address': '731 Valmar Mall', 'city': 'Emporia'},
  'credit-card': {'number': '2241 6061 4754 2939',
   'expiration-date': '12/25'}},
 {'age': 45,
  'name': ['Hershel', 'Montoya'],
  'occupation': 'Health And Safety',
  'telephone': '1-544-536-4535',
  'address': {'address': '391 Villa Circle', 'city': 'Kissimmee'},
  'credit-card': {'number': '4248 5447 2062 7406',
   'expiration-date': '02/23'}})

In [29]:
b.map(lambda record: record['occupation']).take(2)  # Select the occupation field


('Bill Poster', 'Show Jumper')

In [30]:
b.count().compute()  # Count total number of records

10000

### Chain computations
It is common to do many of these steps in one pipeline, only calling compute or take at the end.

In [35]:
result = (b.filter(lambda record: record['age'] > 30)
           .map(lambda record: record['occupation'])
           .frequencies(sort=True)
           .topk(10, key=1))
result

dask.bag<topk-aggregate, npartitions=1>

In [38]:
result.compute()
result.count().compute()

[('Repair Man', 18),
 ('Chandler', 15),
 ('Litigation Manager', 14),
 ('Registrar', 14),
 ('Public House Manager', 14),
 ('Horse Dealer', 13),
 ('Reflexologist', 13),
 ('Waitress', 13),
 ('Publicity Manager', 13),
 ('Merchant', 13)]

10

### Transform and Store
Sometimes we want to compute aggregations as above, but sometimes we want to store results to disk for future analyses. For that we can use methods like to_textfiles and json.dumps, or we can convert to Dask Dataframes and use their storage systems, which we’ll see more of in the next section.

In [39]:
(b.filter(lambda record: record['age'] > 30)  # Select records of interest
  .map(json.dumps)                            # Convert Python objects to text
  .to_textfiles('data/processed.*.json'))     # Write to local disk

['/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/processed.0.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/processed.1.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/processed.2.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/processed.3.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/processed.4.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/processed.5.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/processed.6.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/parallel computing /DASK/data/processed.7.json',
 '/Users/parsanemati/Yandex.Disk.localized/github/data_science_eda/paral

2022-10-13 23:45:38,506 - tornado.application - ERROR - Uncaught exception GET /groups/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/groups/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/Users/parsanemati/.local/lib/python3.8/site-packages/tornado/websocket.py", line 942, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "/Users/parsanemati/.local/lib/python3.8/site-packages/tornado/web.py", line 3208, in wrapper
    return method(self, *args, **kwargs)
  File "/Users/parsanemati/Library/r-miniconda-arm64/envs/nlp/lib/python3.8/site-packages/bokeh/server/views/ws.py", line 149, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.


### Convert to Dask Dataframes
Dask Bags are good for reading in initial data, doing a bit of pre-processing, and then handing off to some other more efficient form like Dask Dataframes. Dask Dataframes use Pandas internally, and so can be much faster on numeric data and also have more complex algorithms.

However, Dask Dataframes also expect data that is organized as flat columns. It does not support nested JSON data very well (Bag is better for this).

Here we make a function to flatten down our nested data structure, map that across our records, and then convert that to a Dask Dataframe.



In [40]:
b.take(1)

({'age': 22,
  'name': ['Roger', 'Whitaker'],
  'occupation': 'Bill Poster',
  'telephone': '309.151.9537',
  'address': {'address': '195 Bergen Stravenue', 'city': 'Batavia'},
  'credit-card': {'number': '4050 7093 7680 9254',
   'expiration-date': '10/22'}},)

In [41]:
def flatten(record):
    return {
        'age': record['age'],
        'occupation': record['occupation'],
        'telephone': record['telephone'],
        'credit-card-number': record['credit-card']['number'],
        'credit-card-expiration': record['credit-card']['expiration-date'],
        'name': ' '.join(record['name']),
        'street-address': record['address']['address'],
        'city': record['address']['city']
    }

b.map(flatten).take(1)

({'age': 22,
  'occupation': 'Bill Poster',
  'telephone': '309.151.9537',
  'credit-card-number': '4050 7093 7680 9254',
  'credit-card-expiration': '10/22',
  'name': 'Roger Whitaker',
  'street-address': '195 Bergen Stravenue',
  'city': 'Batavia'},)

In [42]:
df = b.map(flatten).to_dataframe()
df.head()

,age,occupation,telephone,credit-card-number,credit-card-expiration,name,street-address,city
0,22,Bill Poster,309.151.9537,4050 7093 7680 9254,10/22,Roger Whitaker,195 Bergen Stravenue,Batavia
1,19,Show Jumper,+1-(701)-259-4365,4187 5843 8083 9670,04/24,Olen Shepard,1229 Finley Stravenue,Laconia
2,29,Publishing Manager,1-793-335-3530,3462 831917 58783,05/19,Art Weeks,1058 Ashburton Freeway,Hialeah
3,19,Buyer,318.741.1741,3406 926915 65959,01/24,Akilah Hatfield,1222 Red Leaf Creek,Dayton
4,45,Artexer,148-805-6650,2241 6061 4754 2939,12/25,Moshe Pope,731 Valmar Mall,Emporia


In [46]:
df[df.age > 30].occupation.value_counts().nlargest(10).compute()
type(df)

Repair Man              18
Chandler                15
Registrar               14
Public House Manager    14
Litigation Manager      14
Horse Dealer            13
Waitress                13
Machine Fitters         13
Publicity Manager       13
Reflexologist           13
Name: occupation, dtype: int64

dask.dataframe.core.DataFrame

## Dask DataFrames

### Create Random Dataframe
We create a random timeseries of data with the following attributes:

- It stores a record for every second in the month of January of the year 2000

- It splits that month by day, keeping each day as a partitioned dataframe

- Along with a datetime index it has columns for names, ids, and numeric values

In [48]:
import dask

df = dask.datasets.timeseries()
df
df.dtypes


,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


id        int64
name     object
x       float64
y       float64
dtype: object

In [49]:
# This sets some formatting parameters for displayed data.
import pandas as pd

pd.options.display.precision = 2
pd.options.display.max_rows = 10

df.head(3)

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,940,Ray,-0.72,0.06
2000-01-01 00:00:01,948,Patricia,-0.19,-0.82
2000-01-01 00:00:02,1008,Jerry,-1.00,-0.79


### Use Standard Pandas Operations


https://examples.dask.org/dataframe.html#Use-Standard-Pandas-Operations